In [ ]:
import numpy as np
import matplotlib.pyplot as plt
    
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Dict

from datasets import load_dataset

import utils as utils
from transformers import AutoProcessor, SeamlessM4TModel, SeamlessM4TForTextToText

from torch.cuda import empty_cache
from transformers import pipeline

import torch

Huggingface models

In [ ]:
model_id = 'facebook/hf-seamless-m4t-large'

In [ ]:
processor = AutoProcessor.from_pretrained(model_id, src_lang="eng")
model = SeamlessM4TForTextToText.from_pretrained(
    model_id,
    device_map='cuda:0',
    torch_dtype=torch.bfloat16
)

Hugginface open datasets

In [ ]:
dataset = load_dataset('HuggingFaceH4/no_robots')

In [ ]:
def get_stringify_conversation(example: Dict):
    str_msg = ""
    for message in example['messages']:
        str_msg += message['content']
        str_msg += '\n'

    example['str_msg'] = str_msg
    return example

In [ ]:
dataset = dataset.map(
    utils.get_stringify_conversation,
    num_proc=20
)

In [ ]:
print(dataset['train_sft']['str_msg'][0])

In [ ]:
m = dataset['train_sft']['str_msg'][0:10]

In [ ]:
text_inputs = processor(text=m, src_lang="eng", return_tensors="pt").input_ids.to('cuda:0')

In [ ]:
model.config.max_new_tokens = 2048

In [ ]:
model.config.max_new_tokens

In [ ]:
%%time
# from text
output_tokens = model.generate(
    text_inputs, tgt_lang="hin"
    # max_new_tokens=2046
)

In [ ]:
len(output_tokens[1].tolist())

In [ ]:
processor.batch_decode(output_tokens, skip_special_tokens=True)#.to('cpu')

In [ ]:
translated_text_from_text = processor.decode(
    output_tokens, skip_special_tokens=False
)

In [ ]:
translated_text_from_text